# На вход threshold, на выход вероятности классов

In [181]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import transforms
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm
import torchsummary
import numpy as np

In [182]:
torch.cuda.is_available()

False

In [246]:
class ThresholdModel(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.lin_layers = nn.Sequential(
            nn.Linear(6, 32),
            nn.ReLU(),
            nn.Linear(32, num_classes),
            nn.Softmax()
        )

    def forward(self, x):
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        x = self.lin_layers(x)

        return x

In [257]:
from IPython.display import clear_output


def run_single(model, features, answer, criterion, optimizer=None, device='cpu', train=True):
    if train:
        model.train()
    else:
        model.eval()

    optimizer.zero_grad() if train else None

    output = model(features)
    loss = criterion(output, answer)

    if train:
        loss.backward()
        optimizer.step()

    return (loss.item(), output)


def plot_losses(train_losses, valid_losses):
    plt.figure(figsize=(10, 5))
    plt.plot(train_losses, label='Train Loss')
    plt.plot(valid_losses, label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    plt.title('Training and Validation Loss Over Epochs')
    plt.show()

In [258]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [285]:
num_classes = 2
model = ThresholdModel(num_classes).to(device)
torchsummary.summary(model, (6,))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 32]             224
              ReLU-2                   [-1, 32]               0
            Linear-3                    [-1, 2]              66
           Softmax-4                    [-1, 2]               0
Total params: 290
Trainable params: 290
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------


In [286]:
dataset = []

with open("color_data.txt") as file:
    for line in file:
        features = torch.tensor(list(map(int, line.split()[:6])), dtype=torch.float32)
        answer = torch.tensor(int(line.split()[6]))
        dataset.append((features, answer))


In [287]:
int(dataset[0][1])

1

In [281]:
step = 1
length = len(dataset)

for i in range(length):
    x = dataset[i]
    L1, L2, A1, A2, B1, B2 = x[0]
    
    for L in range(int(L1) + step, int(L2), step):
        newx = torch.tensor(x[0])
        newx[1] = L
        dataset.append((newx, x[1]))

        newx = torch.tensor(x[0])
        newx[0] = L
        dataset.append((newx, x[1]))

    for A in range(int(A1) + step, int(A2), step):
        newx = x[0]
        newx[3] = A
        dataset.append((newx, x[1]))

        newx = x[0]
        newx[2] = L
        dataset.append((newx, x[1]))

    for B in range(int(B1) + step, int(B2), step):
        newx = x[0]
        newx[5] = B
        dataset.append((newx, x[1]))

        newx = x[0]
        newx[4] = B
        dataset.append((newx, x[1]))

C:\Users\Игорь\AppData\Local\Temp\ipykernel_7548\1976583034.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  newx = torch.tensor(x[0])
C:\Users\Игорь\AppData\Local\Temp\ipykernel_7548\1976583034.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  newx = torch.tensor(x[0])


In [288]:
len(dataset)

2

In [289]:
for i in range(5):
    for i in range(len(dataset)):
        loss, answer = run_single(model, dataset[i][0], dataset[i][1], criterion=criterion, optimizer=optimizer, device=device, train=True)
        print(loss)

0.31326165795326233
1.31326162815094


In [290]:
for i in range(len(dataset)):
    # print(dataset[i][0], int(dataset[i][1]))
    loss, pred = run_single(model, dataset[i][0], dataset[i][1], criterion=criterion, optimizer=optimizer, device=device, train=False)
    print(int(torch.argmax(pred)), int(dataset[i][1]), pred)

1 1 tensor([2.3341e-09, 1.0000e+00], grad_fn=<SoftmaxBackward0>)
1 0 tensor([3.2242e-11, 1.0000e+00], grad_fn=<SoftmaxBackward0>)
